In [1]:
import pandas as pd

In [2]:
%pwd

'C:\\Users\\peter\\Documents\\Projekte\\WohnenSG\\EDA'

In [3]:
df = pd.read_csv('belegung-neu-erstellter-wohnungen-2011-2019.csv',  sep=";")

df = pd.read_csv('belegung-neu-erstellter-wohnungen-2011-2019.csv',  sep=";", parse_dates=['Monat_Jahr'])
df=df.drop(df.columns[0:1], axis=1)

In [4]:
df.head()

,EGID_chiff,EWID,Belegungsjahr,Bauperiode,Quartiersgruppe,Quartiersgruppe Name,Wohnungsgroesse,Anz_Kinder,Anz_Erwachsene,HH_Typ,HH_Typ_label,Erstellungsjahr
0,0-163-57-186-165-159-122-82-174-60-177-232-38-...,1.0,2017,8022,13,Quartiergruppe Lachen,4,0,2,5,Haushalte mit zwei Erwachsenen,2014
1,0-179-129-113-45-253-25-61-23-233-133-131-62-7...,2.0,2019,8023,13,Quartiergruppe Lachen,5,0,2,5,Haushalte mit zwei Erwachsenen,2016
2,0-179-129-113-45-253-25-61-23-233-133-131-62-7...,3.0,2019,8023,13,Quartiergruppe Lachen,5,0,1,1,Einpersonenhaushalt,2016
3,0-179-129-113-45-253-25-61-23-233-133-131-62-7...,4.0,2019,8023,13,Quartiergruppe Lachen,3,0,2,5,Haushalte mit zwei Erwachsenen,2016
4,101-71-97-225-239-65-32-208-5-226-231-118-63-2...,1.0,2016,8022,13,Quartiergruppe Lachen,4,0,1,1,Einpersonenhaushalt,2013


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11025 entries, 0 to 11024
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   EGID_chiff            11025 non-null  object 
 1   EWID                  11016 non-null  float64
 2   Belegungsjahr         11025 non-null  int64  
 3   Bauperiode            11025 non-null  int64  
 4   Quartiersgruppe       11025 non-null  int64  
 5   Quartiersgruppe Name  11025 non-null  object 
 6   Wohnungsgroesse       11025 non-null  int64  
 7   Anz_Kinder            11025 non-null  int64  
 8   Anz_Erwachsene        11025 non-null  int64  
 9   HH_Typ                11025 non-null  int64  
 10  HH_Typ_label          11025 non-null  object 
 11  Erstellungsjahr       11025 non-null  int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 1.0+ MB


In [6]:
df.describe()

,EWID,Belegungsjahr,Bauperiode,Quartiersgruppe,Wohnungsgroesse,Anz_Kinder,Anz_Erwachsene,HH_Typ,Erstellungsjahr
count,11016.000000,11025.000000,11025.000000,11025.000000,11025.000000,11025.000000,11025.000000,11025.000000,11025.000000
mean,6.778867,2016.078186,8022.106576,21.442812,3.354649,0.253424,1.706395,3.190023,2012.925351
std,14.556264,2.318612,0.308587,9.411937,1.042951,0.617971,0.688821,1.892879,2.087169
min,1.000000,2011.000000,8022.000000,11.000000,1.000000,0.000000,0.000000,1.000000,2011.000000
25%,3.000000,2014.000000,8022.000000,12.000000,3.000000,0.000000,1.000000,1.000000,2011.000000
50%,5.000000,2016.000000,8022.000000,21.000000,3.000000,0.000000,2.000000,3.000000,2012.000000
75%,9.000000,2018.000000,8022.000000,32.000000,4.000000,0.000000,2.000000,5.000000,2014.000000
max,999.000000,2019.000000,8023.000000,35.000000,10.000000,5.000000,10.000000,7.000000,2019.000000


In [7]:
# Combine 'EGID_chiff' and 'EWID' into a new column named 'ID'
df['ID'] = df['EGID_chiff'].astype(str) + '-' + df['EWID'].astype(str)

# Reorder the columns to place 'Building_Apartment_ID' after 'EGID_chiff'
column_order = ['EGID_chiff', 'EWID', 'ID'] + [col for col in df.columns if col not in ['EGID_chiff', 'EWID', 'ID']]
df = df[column_order]

### Ableitung neuer Spalten für Auswertung

In [8]:
# First, we'll sort the data by 'Building_Apartment_ID' and 'Belegungsjahr' to prepare for change detection.
df_sorted = df.sort_values(by=['ID', 'Belegungsjahr'])


In [9]:
df_sorted['Wechsel'] = (
    (df_sorted['HH_Typ'] != df_sorted['HH_Typ'].shift())
) & (df_sorted['ID'] == df_sorted['ID'].shift()) & (df_sorted['Belegungsjahr'] != df_sorted['Belegungsjahr'].shift()) & (df_sorted['Belegungsjahr'] != df_sorted['Belegungsjahr'].min())

# Since we're interested in changes within the same apartment, we need to ensure that the shift operation
# doesn't mix different apartments, hence the condition on 'ID'. As well, the "move in" is not considered a change!

In [10]:
df_sorted['Dichte'] =  (df_sorted['Anz_Erwachsene'] + 0.5 * df_sorted['Anz_Kinder']) / df_sorted['Wohnungsgroesse']

In [23]:
df_sorted['Ueberbelegung'] = (df_sorted['Anz_Erwachsene'] + df_sorted['Anz_Kinder']) >= (df_sorted['Wohnungsgroesse'] + 2)

#### Es gibt so wenig Wohnungen mit 6 Zimmern oder mehr, dass ich diese zusammen fasse

In [12]:
df_sorted['WGM'] = df_sorted['Wohnungsgroesse'].apply(lambda x: '6+' if x >= 6 else x)

#### Bereinigung langer Namen

In [13]:
df_sorted['Quartiersgruppe Name'] = df_sorted['Quartiersgruppe Name'].str.replace('Quartiergruppe ', '')

In [14]:
df_sorted['Quartiersgruppe Name'] = df_sorted['Quartiersgruppe Name'].replace('Linsebühl - Dreilinden', 'Linsebühl')

In [15]:
df_sorted['Quartiersgruppe Name'] = df_sorted['Quartiersgruppe Name'].replace('Langgass - Heiligkreuz', 'Langgass')

In [16]:
df_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11025 entries, 2240 to 2220
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   EGID_chiff            11025 non-null  object 
 1   EWID                  11016 non-null  float64
 2   ID                    11025 non-null  object 
 3   Belegungsjahr         11025 non-null  int64  
 4   Bauperiode            11025 non-null  int64  
 5   Quartiersgruppe       11025 non-null  int64  
 6   Quartiersgruppe Name  11025 non-null  object 
 7   Wohnungsgroesse       11025 non-null  int64  
 8   Anz_Kinder            11025 non-null  int64  
 9   Anz_Erwachsene        11025 non-null  int64  
 10  HH_Typ                11025 non-null  int64  
 11  HH_Typ_label          11025 non-null  object 
 12  Erstellungsjahr       11025 non-null  int64  
 13  Wechsel               11025 non-null  bool   
 14  Dichte                11025 non-null  float64
 15  Ueberbelegung         

### Test Erstellung von Matrizen mit .groupby and .pivot

In [17]:
df_grouped = df_sorted.groupby(['Quartiersgruppe Name', 'WGM'])['Wechsel'].mean().reset_index()

In [18]:
df_grouped

,Quartiersgruppe Name,WGM,Wechsel
0,Bruggen,1,0.000000
1,Bruggen,2,0.102339
2,Bruggen,3,0.156436
3,Bruggen,4,0.141782
4,Bruggen,5,0.149123
...,...,...,...
70,Winkeln,2,0.071429
71,Winkeln,3,0.145522
72,Winkeln,4,0.179528
73,Winkeln,5,0.166667


In [19]:
df_table = df_grouped.pivot(index='WGM', columns='Quartiersgruppe Name', values='Wechsel')

In [20]:
df_table

Quartiersgruppe Name,Bruggen,Innenstadt,Lachen,Langgass,Linsebühl,Neudorf,Notkersegg,Riethüsli,Rosenberg,Rotmonten,St.Fiden,St.Georgen,St.Jakob,Winkeln
WGM,,,,,,,,,,,,,,
1,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.057692,0.037037,0.000000,0.000000,NaN,0.000000
2,0.102339,0.076923,0.121622,0.121302,0.021739,0.088435,0.0,0.065574,0.056180,0.105263,0.075949,0.000000,0.000000,0.071429
3,0.156436,0.000000,0.184314,0.142329,0.180000,0.174515,0.0,0.143885,0.142500,0.071429,0.117647,0.028571,0.142857,0.145522
4,0.141782,0.000000,0.194888,0.149378,0.147541,0.156398,0.0,0.118644,0.159292,0.082645,0.140187,0.068966,0.200000,0.179528
5,0.149123,NaN,0.108434,0.230769,0.096774,0.171875,NaN,0.065217,0.000000,0.045455,0.130841,0.125000,0.000000,0.166667
6+,0.000000,0.000000,0.000000,0.090909,NaN,0.052632,NaN,0.000000,0.000000,0.055556,0.088889,0.000000,0.000000,0.266667


In [21]:
df_table = df_table.reset_index()

In [22]:
df_table

Quartiersgruppe Name,WGM,Bruggen,Innenstadt,Lachen,Langgass,Linsebühl,Neudorf,Notkersegg,Riethüsli,Rosenberg,Rotmonten,St.Fiden,St.Georgen,St.Jakob,Winkeln
0,1,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.057692,0.037037,0.000000,0.000000,NaN,0.000000
1,2,0.102339,0.076923,0.121622,0.121302,0.021739,0.088435,0.0,0.065574,0.056180,0.105263,0.075949,0.000000,0.000000,0.071429
2,3,0.156436,0.000000,0.184314,0.142329,0.180000,0.174515,0.0,0.143885,0.142500,0.071429,0.117647,0.028571,0.142857,0.145522
3,4,0.141782,0.000000,0.194888,0.149378,0.147541,0.156398,0.0,0.118644,0.159292,0.082645,0.140187,0.068966,0.200000,0.179528
4,5,0.149123,NaN,0.108434,0.230769,0.096774,0.171875,NaN,0.065217,0.000000,0.045455,0.130841,0.125000,0.000000,0.166667
5,6+,0.000000,0.000000,0.000000,0.090909,NaN,0.052632,NaN,0.000000,0.000000,0.055556,0.088889,0.000000,0.000000,0.266667


In [37]:
rowData=df_table.to_dict("records")
rowData

[{'WGM': 1,
  'Bruggen': 0.0,
  'Innenstadt': nan,
  'Lachen': 0.0,
  'Langgass': 0.0,
  'Linsebühl': 0.0,
  'Neudorf': nan,
  'Notkersegg': nan,
  'Riethüsli': nan,
  'Rosenberg': 0.057692307692307696,
  'Rotmonten': 0.037037037037037035,
  'St.Fiden': 0.0,
  'St.Georgen': 0.0,
  'St.Jakob': nan,
  'Winkeln': 0.0},
 {'WGM': 2,
  'Bruggen': 0.1023391812865497,
  'Innenstadt': 0.07692307692307693,
  'Lachen': 0.12162162162162163,
  'Langgass': 0.12130177514792899,
  'Linsebühl': 0.021739130434782608,
  'Neudorf': 0.08843537414965986,
  'Notkersegg': 0.0,
  'Riethüsli': 0.06557377049180328,
  'Rosenberg': 0.056179775280898875,
  'Rotmonten': 0.10526315789473684,
  'St.Fiden': 0.0759493670886076,
  'St.Georgen': 0.0,
  'St.Jakob': 0.0,
  'Winkeln': 0.07142857142857142},
 {'WGM': 3,
  'Bruggen': 0.15643564356435644,
  'Innenstadt': 0.0,
  'Lachen': 0.1843137254901961,
  'Langgass': 0.1423290203327172,
  'Linsebühl': 0.18,
  'Neudorf': 0.1745152354570637,
  'Notkersegg': 0.0,
  'Riethüsli':

#### Test ok, also Download des DF für die App

In [24]:
df_sorted.to_csv('df.csv', index=False)

#### Und noch ein kleiner Exkurs, wie das mit Lists von Dicts so funktioniert...

In [24]:
df.columns

Index(['EGID_chiff', 'EWID', 'ID', 'Belegungsjahr', 'Bauperiode',
       'Quartiersgruppe', 'Quartiersgruppe Name', 'Wohnungsgroesse',
       'Anz_Kinder', 'Anz_Erwachsene', 'HH_Typ', 'HH_Typ_label',
       'Erstellungsjahr'],
      dtype='object')

In [32]:
 columnDefs=[{"field": "WGM"}, {"field": i, "valueFormatter": {"function": "d3.format(',.1%')(params.value)"}} for i in df_table.columns[1:]]

SyntaxError: did you forget parentheses around the comprehension target? (1109436615.py, line 1)

In [34]:
columnDefs = [{"field": "WGM"}]+ columnDefs

In [35]:
columnDefs

[{'field': 'WGM'},
 {'field': 'Bruggen',
  'valueFormatter': {'function': "d3.format(',.1%')(params.value)"}},
 {'field': 'Innenstadt',
  'valueFormatter': {'function': "d3.format(',.1%')(params.value)"}},
 {'field': 'Lachen',
  'valueFormatter': {'function': "d3.format(',.1%')(params.value)"}},
 {'field': 'Langgass',
  'valueFormatter': {'function': "d3.format(',.1%')(params.value)"}},
 {'field': 'Linsebühl',
  'valueFormatter': {'function': "d3.format(',.1%')(params.value)"}},
 {'field': 'Neudorf',
  'valueFormatter': {'function': "d3.format(',.1%')(params.value)"}},
 {'field': 'Notkersegg',
  'valueFormatter': {'function': "d3.format(',.1%')(params.value)"}},
 {'field': 'Riethüsli',
  'valueFormatter': {'function': "d3.format(',.1%')(params.value)"}},
 {'field': 'Rosenberg',
  'valueFormatter': {'function': "d3.format(',.1%')(params.value)"}},
 {'field': 'Rotmonten',
  'valueFormatter': {'function': "d3.format(',.1%')(params.value)"}},
 {'field': 'St.Fiden',
  'valueFormatter': {'fu